In [14]:
import os
import pandas as pd

# Define the directory where the Excel files are stored
folder_path = r'Dataset\train'

In [15]:
dataframes = []


In [16]:
# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Adjust the file extension if needed
        file_path = os.path.join(folder_path, filename)
        # Read the Excel file into a dataframe
        df = pd.read_excel(file_path)
        dataframes.append(df)  # Add each dataframe to the list

ValueError: Excel file format cannot be determined, you must specify an engine manually.